<div style='float: right'><img src='pic/abcplace.png'/></div>

## ABCプレース

### 問題
* 各行各列にA、B、Cを1つずつ置く。
* 外側のヒントは、最も近い文字を表す。

### 変数
* v：各文字 (1)

### 制約
* ヒントを満たす (2)
* 各行各列に1つ (3)
* 文字は1つ (4)

In [1]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
from ortools.sat.python import cp_model
data = """\
 B B  C 
A......B
B...... 
C...... 
 ......A
 ......C
 ...... 
  BC CA """.split('\n')
nn = len(data)-2

In [2]:
def chk(v, c, isless):
    k = ord(c) - 65
    if k >= 0:
        for j in range(3):
            if j != k:
                m.Add(v[k] < v[j] if isless else v[k] > v[j])
m = cp_model.CpModel()
vh = np.array([[m.NewIntVar(0, nn - 1, f'vh{i}{j}') for j in range(3)] for i in range(nn)])
vv = np.array([[m.NewIntVar(0, nn - 1, f'vv{i}{j}') for j in range(3)] for i in range(nn)])
for i in range(nn):
    m.AddAllDifferent(vh[i])
    m.AddAllDifferent(vv[i])
    chk(vv[i], data[0][i + 1], True)
    chk(vv[i], data[nn + 1][i + 1], False)
    chk(vh[i], data[i + 1][0], True)
    chk(vh[i], data[i + 1][nn + 1], False)
for i, j, k in product(range(nn), range(nn), range(3)):
    vb1, vb2 = m.NewBoolVar(f'vb1{i}{j}{k}'), m.NewBoolVar(f'vb2{i}{j}{k}')
    m.Add(vh[i][k] == j).OnlyEnforceIf(vb1)
    m.Add(vv[j][k] != i).OnlyEnforceIf(vb2)
    m.AddBoolXOr([vb1, vb2])
solver = cp_model.CpSolver()
%time status = solver.Solve(m)
res = np.vectorize(solver.Value)(vh)
cc = np.full((nn, nn), '.')
for i, j in product(range(nn), range(3)):
    cc[i][res[i][j]] = chr(65 + j)
print('\n'.join(' '.join(s) for s in cc))

CPU times: user 70 ms, sys: 10 ms, total: 80 ms
Wall time: 78.1 ms
. A . C B .
. . B A . C
. C A . . B
B . C . A .
A B . . C .
C . . B . A


In [3]:
m = LpProblem()
v = np.array(addbinvars(nn, nn, 4)) # (1)
def chk(m, c, y, x, dy, dx):
    d = ord(c)-64
    if d > 0:
        for i, k in product(range(1,nn), range(1,4)):
            if k != d:
                for j in range(i):
                    m += v[y+dy*j,x+dx*j,k] <= 1-v[y+dy*i,x+dx*i,d] # (2)
for i in range(nn):
    for k in range(1, 4):
        m += lpSum(v[i,:,k]) == 1 # (3)
        m += lpSum(v[:,i,k]) == 1 # (3)
    chk(m, data[0][i+1], 0, i, 1, 0)
    chk(m, data[nn+1][i+1], nn-1, i, -1, 0)
    chk(m, data[i+1][0], i, 0, 0, 1)
    chk(m, data[i+1][nn+1], i, nn-1, 0, -1)
for i, j in product(range(nn), range(nn)):
    m += lpSum(v[i,j]) == 1 # (4)
%time m.solve()
print('\n'.join(' '.join('.ABC'[i] for i in j)
    for j in np.vectorize(value)(v).dot(range(4)).astype(int)))

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 40.8 ms
. A . C B .
. . B A . C
. C A . . B
B . C . A .
A B . . C .
C . . B . A
